In [1]:
!git clone https://github.com/hungpham13/federated-learning
%cd federated-learning
!pip install -q -r requirements.txt

Cloning into 'federated-learning'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 55 (delta 28), reused 35 (delta 14), pack-reused 0
Unpacking objects: 100% (55/55), 66.06 KiB | 2.20 MiB/s, done.
/content/federated-learning
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade --index-url https://pypi.ngc.nvidia.com nvidia-tensorrt

(pid=2458) 2023-06-09 12:47:10.711192: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Looking in indexes: https://pypi.ngc.nvidia.com, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.ngc.nvidia.com


In [2]:
import flwr as fl
from config import NUM_CLIENTS, DEVICE
from model import Net, VGG16
from main import simulate_cifar

## Centralize training

In [ ]:
from dataloader import load_cifars
trainloaders, valloaders, testloader = load_cifars(1)

trainloader = trainloaders[0]
valloader = valloaders[0]
net = VGG16().to(DEVICE)

for epoch in range(10):
    net.train_epoch(trainloader, 1)
    loss, accuracy = net.test(valloader)
    print(f"Epoch {epoch+1}: validation loss {loss}, accuracy {accuracy}")

loss, accuracy = net.test(testloader)
print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

Files already downloaded and verified
Files already downloaded and verified
Epoch 1: train loss 0.05028054490685463, accuracy 0.3868888888888889
Epoch 1: validation loss 0.04147230490446091, accuracy 0.5456
Epoch 1: train loss 0.033639177680015564, accuracy 0.6148
Epoch 2: validation loss 0.026561914348602295, accuracy 0.7034
Epoch 1: train loss 0.02576061710715294, accuracy 0.7148666666666667
Epoch 3: validation loss 0.02317628452181816, accuracy 0.744
Epoch 1: train loss 0.020628293976187706, accuracy 0.7752222222222223
Epoch 4: validation loss 0.021399199867248534, accuracy 0.771
Epoch 1: train loss 0.017199356108903885, accuracy 0.8127111111111112
Epoch 5: validation loss 0.01969984709620476, accuracy 0.7902
Epoch 1: train loss 0.0143361184746027, accuracy 0.8449333333333333
Epoch 6: validation loss 0.019189575929939747, accuracy 0.8004
Epoch 1: train loss 0.012040548026561737, accuracy 0.8689333333333333
Epoch 7: validation loss 0.017127903091907502, accuracy 0.8216
Epoch 1: train

## Fed Average

In [3]:
strategyConfig = {
    "fraction_fit":0.3,
    "fraction_evaluate":0.3,
    "min_fit_clients":3,
    "min_evaluate_clients":3,
    "min_available_clients":NUM_CLIENTS,
}

In [ ]:
net = Net().to(DEVICE)
simulate_cifar(StrategyCls=fl.server.strategy.FedAvg, strategyArgs=strategyConfig, net=net, num_rounds=50)

100%|██████████| 170498071/170498071 [00:04<00:00, 34372272.94it/s]


Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


INFO flwr 2023-06-10 08:59:06,094 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=100, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=100, round_timeout=None)
2023-06-10 08:59:09,323	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-10 08:59:10,858 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7861176731.0, 'object_store_memory': 3930588364.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7861176731.0, 'object_store_memory': 3930588364.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-06-10 08:59:10,867 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-10 08:59:10,875 | server.py:269 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023-06-10 08:59:10,883 | server.py:88

Server-side evaluation loss 0.07212100248336792 / accuracy 0.1


(pid=1227) 2023-06-10 08:59:29.365097: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


...

(launch_and_fit pid=38989) Epoch 2: train loss 0.009492313489317894, accuracy 0.9024444444444445


INFO flwr 2023-06-10 10:22:23,523 | server.py:119 | fit progress: (74, 0.05235792600214481, {'accuracy': 0.5966}, 4981.523183888)
INFO:flwr:fit progress: (74, 0.05235792600214481, {'accuracy': 0.5966}, 4981.523183888)
DEBUG flwr 2023-06-10 10:22:23,528 | server.py:168 | evaluate_round 74: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 74: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.05235792600214481 / accuracy 0.5966


(pid=39115) 2023-06-10 10:22:29.445079: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=39115) [Client 1] evaluate, config: {}


In [ ]:
net = VGG16().to(DEVICE)
simulate_cifar(StrategyCls=fl.server.strategy.FedAvg, strategyArgs=strategyConfig, net=net, num_rounds=50)

## Fed Ada Grad

In [7]:
net = Net().to(DEVICE)
simulate_cifar(StrategyCls=fl.server.strategy.FedAdagrad, strategyArgs=strategyConfig, net=net, num_rounds=10)

Files already downloaded and verified
Files already downloaded and verified


INFO flwr 2023-06-10 04:42:56,677 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-06-10 04:43:03,944	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-10 04:43:05,946 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'object_store_memory': 3935900467.0, 'memory': 7871800935.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'object_store_memory': 3935900467.0, 'memory': 7871800935.0}
INFO flwr 2023-06-10 04:43:05,955 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-10 04:43:05,958 | server.py:269 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023-06-10 04:43:05,963 | server.py:88 |

Server-side evaluation loss 0.0721333030462265 / accuracy 0.1001


(pid=15396) 2023-06-10 04:43:15.245406: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_fit pid=15396) [Client 7, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=15396) Epoch 1: train loss 0.0641552060842514, accuracy 0.22822222222222222


...

(pid=20328) 2023-06-10 04:53:36.719354: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=20328) [Client 0] evaluate, config: {}


(pid=20398) 2023-06-10 04:53:44.643925: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=20398) [Client 3] evaluate, config: {}


DEBUG flwr 2023-06-10 04:53:48,616 | server.py:182 | evaluate_round 10 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 3 results and 0 failures
INFO flwr 2023-06-10 04:53:48,620 | server.py:147 | FL finished in 638.8713389230002
INFO:flwr:FL finished in 638.8713389230002
INFO flwr 2023-06-10 04:53:48,621 | app.py:218 | app_fit: losses_distributed [(1, 5.221945373535156), (2, 0.2953500607808431), (3, 0.16375938351949057), (4, 0.19685283088684083), (5, 0.14276635980606078), (6, 0.09279807392756144), (7, 0.07258106629053751), (8, 0.07318584672609967), (9, 0.056748929421106965), (10, 0.056705404996871954)]
INFO:flwr:app_fit: losses_distributed [(1, 5.221945373535156), (2, 0.2953500607808431), (3, 0.16375938351949057), (4, 0.19685283088684083), (5, 0.14276635980606078), (6, 0.09279807392756144), (7, 0.07258106629053751), (8, 0.07318584672609967), (9, 0.056748929421106965), (10, 0.056705404996871954)]
INFO flwr 2023-06-10 04:53:48,624 | app.py:219 | app_fit: metrics_d

In [8]:
net = VGG16().to(DEVICE)
simulate_cifar(StrategyCls=fl.server.strategy.FedAdagrad, strategyArgs=strategyConfig, net=net, num_rounds=10)

Files already downloaded and verified
Files already downloaded and verified


INFO flwr 2023-06-10 04:53:50,380 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-06-10 04:53:56,544	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-10 04:53:57,952 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7870788404.0, 'object_store_memory': 3935394201.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7870788404.0, 'object_store_memory': 3935394201.0}
INFO flwr 2023-06-10 04:53:57,956 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-10 04:53:57,958 | server.py:269 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023-06-10 04:53:57,960 | server.py:88 |

Server-side evaluation loss 0.07208328914642334 / accuracy 0.1


(pid=20745) 2023-06-10 04:54:11.740887: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_fit pid=20745) [Client 0, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=20745) Epoch 1: train loss 0.06614630669355392, accuracy 0.18977777777777777


...

(pid=26829) 2023-06-10 05:08:51.827368: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=26829) [Client 4] evaluate, config: {}


(pid=26891) 2023-06-10 05:08:59.949090: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=26891) [Client 2] evaluate, config: {}


DEBUG flwr 2023-06-10 05:09:04,812 | server.py:182 | evaluate_round 10 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 3 results and 0 failures
INFO flwr 2023-06-10 05:09:04,817 | server.py:147 | FL finished in 901.814400534
INFO:flwr:FL finished in 901.814400534
INFO flwr 2023-06-10 05:09:04,824 | app.py:218 | app_fit: losses_distributed [(1, 881257811891.5414), (2, 7.139457339709902e+18), (3, 1.2841175805899162e+18), (4, 4.4439478142326717e+18), (5, 2.520041685343316e+19), (6, 4.5572655408770335e+18), (7, 7.97924403428267e+17), (8, 2.6499453795261297e+18), (9, 7.693934957904874e+17), (10, 1.7723267181851548e+18)]
INFO:flwr:app_fit: losses_distributed [(1, 881257811891.5414), (2, 7.139457339709902e+18), (3, 1.2841175805899162e+18), (4, 4.4439478142326717e+18), (5, 2.520041685343316e+19), (6, 4.5572655408770335e+18), (7, 7.97924403428267e+17), (8, 2.6499453795261297e+18), (9, 7.693934957904874e+17), (10, 1.7723267181851548e+18)]
INFO flwr 2023-06-10 05:09:04,827

## q-FFL

In [9]:
net = Net().to(DEVICE)
simulate_cifar(StrategyCls=fl.server.strategy.QFedAvg, strategyArgs=strategyConfig, net=net, num_rounds=10)

Files already downloaded and verified
Files already downloaded and verified


INFO flwr 2023-06-10 05:09:06,390 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-06-10 05:09:13,847	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-10 05:09:15,827 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7872452199.0, 'object_store_memory': 3936226099.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7872452199.0, 'object_store_memory': 3936226099.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0}
INFO flwr 2023-06-10 05:09:15,845 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-10 05:09:15,848 | server.py:269 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023-06-10 05:09:15,854 | server.py:88 |

Server-side evaluation loss 0.07215865316390992 / accuracy 0.0998


(pid=27243) 2023-06-10 05:09:25.180133: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_fit pid=27243) [Client 3, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=27243) Epoch 1: train loss 0.06371299177408218, accuracy 0.22377777777777777


...

(pid=32335) 2023-06-10 05:20:22.592647: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=32335) [Client 1] evaluate, config: {}


(pid=32399) 2023-06-10 05:20:30.991730: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=32399) [Client 7] evaluate, config: {}


DEBUG flwr 2023-06-10 05:20:34,886 | server.py:182 | evaluate_round 10 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 3 results and 0 failures
INFO flwr 2023-06-10 05:20:34,891 | server.py:147 | FL finished in 675.190916773
INFO:flwr:FL finished in 675.190916773
INFO flwr 2023-06-10 05:20:34,897 | app.py:218 | app_fit: losses_distributed [(1, 0.07375088644027711), (2, 0.07375187953313193), (3, 0.07376949659983317), (4, 0.07387237819035848), (5, 0.07364813089370727), (6, 0.0736991367340088), (7, 0.07381131283442179), (8, 0.07367741918563843), (9, 0.0738088657061259), (10, 0.07370338344573975)]
INFO:flwr:app_fit: losses_distributed [(1, 0.07375088644027711), (2, 0.07375187953313193), (3, 0.07376949659983317), (4, 0.07387237819035848), (5, 0.07364813089370727), (6, 0.0736991367340088), (7, 0.07381131283442179), (8, 0.07367741918563843), (9, 0.0738088657061259), (10, 0.07370338344573975)]
INFO flwr 2023-06-10 05:20:34,904 | app.py:219 | app_fit: metrics_distributed

In [10]:
net = VGG16().to(DEVICE)
simulate_cifar(StrategyCls=fl.server.strategy.QFedAvg, strategyArgs=strategyConfig, net=net, num_rounds=10)

Files already downloaded and verified
Files already downloaded and verified


INFO flwr 2023-06-10 05:20:36,652 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-06-10 05:20:44,566	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-10 05:20:46,347 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 7871818139.0, 'object_store_memory': 3935909068.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 7871818139.0, 'object_store_memory': 3935909068.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO flwr 2023-06-10 05:20:46,352 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-10 05:20:46,353 | server.py:269 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023-06-10 05:20:46,355 | server.py:88 |

Server-side evaluation loss 0.07208232378959656 / accuracy 0.1


(pid=32754) 2023-06-10 05:20:58.670965: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_fit pid=32754) [Client 2, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=32754) Epoch 1: train loss 0.06685440242290497, accuracy 0.18533333333333332


...

(pid=39135) 2023-06-10 05:37:06.711996: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=39135) [Client 0] evaluate, config: {}


(pid=39198) 2023-06-10 05:37:14.859724: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=39198) [Client 9] evaluate, config: {}


DEBUG flwr 2023-06-10 05:37:19,684 | server.py:182 | evaluate_round 10 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 3 results and 0 failures
INFO flwr 2023-06-10 05:37:19,689 | server.py:147 | FL finished in 988.1454762120002
INFO:flwr:FL finished in 988.1454762120002
INFO flwr 2023-06-10 05:37:19,700 | app.py:218 | app_fit: losses_distributed [(1, 0.07369879515965778), (2, 0.07367929124832152), (3, 0.07369392855962119), (4, 0.07368517239888508), (5, 0.07367878675460816), (6, 0.07369879611333212), (7, 0.07367650779088337), (8, 0.07366254790623983), (9, 0.07369060309727987), (10, 0.0736981004079183)]
INFO:flwr:app_fit: losses_distributed [(1, 0.07369879515965778), (2, 0.07367929124832152), (3, 0.07369392855962119), (4, 0.07368517239888508), (5, 0.07367878675460816), (6, 0.07369879611333212), (7, 0.07367650779088337), (8, 0.07366254790623983), (9, 0.07369060309727987), (10, 0.0736981004079183)]
INFO flwr 2023-06-10 05:37:19,707 | app.py:219 | app_fit: metrics_d